In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# NLTK tools for text processing
import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


import string
import spacy #used spacy for text prepocessing
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis #import pyLDAvis.gensim no longer works
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\helen\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data = pd.read_csv('yelp_edmonton_reviews_restaurant_categories.csv')
data

C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,business_id,name,address,city,postal_code,latitude,longitude,stars,attributes,categories,user_id,review_stars,text,date
0,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",xTlqpuuqtO0FWs12gd1v6A,5,Came here to print off some documents and was ...,2014-10-15 01:49:26
1,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",f9lq4KAus-xCsmJmjXeKVw,3,Naked Cyber & Expresso Bar is a 24 hours cyber...,2010-01-26 21:14:29
2,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",r3QexFIhBXBT99canAgVEg,4,I visited Naked for the first time this weeken...,2013-02-05 17:01:52
3,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",xjafuSH_qDHXEkU0LyAxvg,4,Neat cafe! Delicious sandwiches and sweets wit...,2017-05-03 16:35:44
4,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",758g6NGLp9deCbvowz62Ww,3,"Naked is a cyber-cafe, one of the first ones t...",2008-10-22 02:56:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56124,Rn3DtOiuJtc3whFySWXs5Q,Japanese Village,2940 Calgary Trail NW,Edmonton,T6J 7C2,53.463917,-113.494702,3.0,"{'RestaurantsAttire': ""u'casual'"", 'NoiseLevel...","Japanese, Restaurants, Steakhouses, Sushi Bars",jHtMpIDrHHVT1F4utXKFXQ,3,"We have been going to Japanese Village (""JV"") ...",2016-05-27 15:33:39
56125,Rn3DtOiuJtc3whFySWXs5Q,Japanese Village,2940 Calgary Trail NW,Edmonton,T6J 7C2,53.463917,-113.494702,3.0,"{'RestaurantsAttire': ""u'casual'"", 'NoiseLevel...","Japanese, Restaurants, Steakhouses, Sushi Bars",X8Z8Tax28_sAsPeNF922jw,2,Went Christmas eve as to celebrate holidays on...,2021-12-25 04:03:42
56126,Rn3DtOiuJtc3whFySWXs5Q,Japanese Village,2940 Calgary Trail NW,Edmonton,T6J 7C2,53.463917,-113.494702,3.0,"{'RestaurantsAttire': ""u'casual'"", 'NoiseLevel...","Japanese, Restaurants, Steakhouses, Sushi Bars",qQxRRzTT0U75A3mTVRh8xw,2,We came here a couple days ago for dinner. It ...,2016-05-16 12:16:46
56127,Rn3DtOiuJtc3whFySWXs5Q,Japanese Village,2940 Calgary Trail NW,Edmonton,T6J 7C2,53.463917,-113.494702,3.0,"{'RestaurantsAttire': ""u'casual'"", 'NoiseLevel...","Japanese, Restaurants, Steakhouses, Sushi Bars",f4B_zY-HuXqW00mWCzaclQ,1,I wouldn't recommend this restaurant. We went ...,2018-04-18 16:44:50


In [113]:
pos_df = data[data.review_stars ==5]
pos_df.info()
# .business_id.nunique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19718 entries, 0 to 56117
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   19718 non-null  object 
 1   name          19718 non-null  object 
 2   address       19673 non-null  object 
 3   city          19718 non-null  object 
 4   postal_code   19712 non-null  object 
 5   latitude      19718 non-null  float64
 6   longitude     19718 non-null  float64
 7   stars         19718 non-null  float64
 8   attributes    19548 non-null  object 
 9   categories    19718 non-null  object 
 10  user_id       19718 non-null  object 
 11  review_stars  19718 non-null  int64  
 12  text          19718 non-null  object 
 13  date          19718 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 2.3+ MB


C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [114]:
neg_df = data[data.review_stars ==1]
neg_df.info()
#.business_id.nunique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5907 entries, 34 to 56127
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   5907 non-null   object 
 1   name          5907 non-null   object 
 2   address       5901 non-null   object 
 3   city          5907 non-null   object 
 4   postal_code   5904 non-null   object 
 5   latitude      5907 non-null   float64
 6   longitude     5907 non-null   float64
 7   stars         5907 non-null   float64
 8   attributes    5866 non-null   object 
 9   categories    5907 non-null   object 
 10  user_id       5907 non-null   object 
 11  review_stars  5907 non-null   int64  
 12  text          5907 non-null   object 
 13  date          5907 non-null   object 
dtypes: float64(3), int64(1), object(10)
memory usage: 692.2+ KB


C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\helen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\helen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [119]:
def wordcount(data):
    #filter data with only single id restaurant
    comments = []
    stop_words = set(stopwords.words('english'))
    #name_col = f'{data.business_id}'
    name_col = data.business_id.unique()
    star_col = data.review_stars.unique()

    for words in data['text']:
        only_letters = re.sub("[^a-zA-Z]", " ",words)
        tokens = nltk.word_tokenize(only_letters) #tokenize the sentences
        lower_case = [l.lower() for l in tokens] #convert all letters to lower case
        filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) #Remove stopwords from the comments
        comments.append(' '.join(filtered_result))
    
    
    w_string = []
    for t in data['text']:
        w_string.append(t)
    w_string = pd.Series(w_string).str.cat(sep=' ')
    
    
    co = CountVectorizer(ngram_range=(2,5))
    counts = co.fit_transform(comments)
    important_trigrams = pd.DataFrame(counts.sum(axis=0),columns=co.get_feature_names()).T.sort_values(0,ascending=False).head(50)
 
    important_trigrams=important_trigrams.reset_index()
    important_trigrams.rename(columns={'index':'word',0:'frequency'},inplace=True)
    important_trigrams['all'] = important_trigrams[['word', 'frequency']].astype(str).apply(lambda x: ' '.join(x), axis=1)
    out_df = important_trigrams[:21]
    out_df = out_df.drop(['word','frequency'], axis = 1)
    out_df_t = out_df.transpose()
    out_df_t['bus_id']= [x for x in name_col]
    out_df_t = out_df_t.astype(str)
    return out_df_t

C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [120]:
wordcount(pos_df[pos_df['business_id'] == 'JX4tUpd09YFchLBuI43lGw'])

C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,bus_id
all,board games 2,access wholly 1,place awesome meet 1,owner coffee smooth delicious well 1,papers printed 1,papers printed great 1,papers printed great customer 1,papers printed great customer service 1,place awesome 1,place awesome meet bob 1,...,place bit 1,place bit hidden 1,place bit hidden clue 1,place bit hidden clue independent 1,owner coffee smooth delicious 1,owner coffee 1,place hang catch 1,open mic enjoyed visit awesome 1,open mic enjoyed visit 1,JX4tUpd09YFchLBuI43lGw


In [123]:
#1935 nunique()
def process():
    df_list = []
    for bus_id in neg_df.business_id.unique():

        df_list.append(wordcount(neg_df[neg_df['business_id'] == bus_id]))

    process_df = pd.concat(df_list)
    process_df.to_csv('out_negative.csv')
    return process_df

C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [124]:
process()

C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,bus_id
all,accordingly going 1,want treat 1,products gotten much worse 1,products gotten much worse past 1,seem want 1,seem want treat 1,seem want treat accordingly 1,seem want treat accordingly going 1,service availability 1,service availability products 1,...,treat accordingly 1,treat accordingly going 1,treat accordingly going back 1,want treat accordingly 1,accordingly going back 1,want treat accordingly going 1,want treat accordingly going back 1,worse past 1,worse past year 1,5BmQX4UVJY19mMtafMg7JA
all,bit whole 1,one even 1,one even worse soon 1,one even worse soon bit 1,pass place 1,pastries sit 1,pastries sit sold 1,pastries sit sold take 1,pastries sit sold take hard 1,rock hard 1,...,rock hard crumbled disappointing theyre 1,sit sold 1,sit sold take 1,sit sold take hard 1,sit sold take hard pass 1,one even worse 1,one chewy crunchy edges indicating 1,sold take hard 1,one chewy crunchy edges 1,JwBTq6IOFH97b9Hr6RT7pw
all,bulk barn 4,licorice laces 2,junk food 2,abundance stock 1,name friend equally 1,must bins 1,must bins knows 1,must bins knows long 1,must bins knows long coupled 1,name friend 1,...,name friend equally luscious load 1,need say 1,need say thoroughly 1,need say thoroughly disappointing 1,need say thoroughly disappointing stale 1,much junk food lol rather 1,much junk 1,much junk food 1,neighbourhood least abundance 1,QU6xxKwHXR6xQ5oi2yIKLw
all,empty restaurant 2,acceptable poor 1,room temperature could understand minutes 1,seemed like piece 1,seemed like 1,scouring pad sure dangerous could 1,scouring pad sure dangerous 1,scouring pad sure 1,scouring pad 1,saw went counter told lady 1,...,saw went 1,room temperature could understand 1,recommend place go back 1,room temperature could 1,room temperature 1,restaurant upset yet started taking 1,restaurant upset yet started 1,restaurant upset yet 1,restaurant upset 1,GpFRwCHN0RI8yFbcfXjfUg
all,absolutely pathetic 1,obnoxious little prick emailed 1,medical exemption simply 1,medical exemption simply exempt 1,medical exemption simply exempt face 1,new line 1,new line steigel 1,new line steigel non 1,new line steigel non alcoholic 1,non alcoholic 1,...,obnoxious little prick 1,obnoxious little prick emailed store 1,prick emailed 1,owner hoping 1,owner hoping resolve 1,owner hoping resolve issue 1,owner hoping resolve issue response 1,pathetic disgrace 1,pathetic disgrace albertans 1,bf34LpGej_MZbOlAKB1beQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
all,pizza would 3,would take 3,customer service 2,pizza good 2,said know 2,fried chicken 2,start quoting 2,pretty good 2,retrieve pen much 1,retrieve pen 1,...,retire something get 1,retrieve pen much effort know 1,retire something 1,reply still time go tell 1,reply still time go 1,reply still time 1,reply still 1,replied yes asked pen laziness 1,replied yes asked pen 1,wKJalp7AabLjVmEgdOIPPw
all,steak house 9,end steak house 6,top end 6,end steak 6,top end steak house 6,top end steak 6,medium rare 5,hate think 3,would hate think 3,would hate 3,...,prime rib 3,ordered medium 3,rib steak 3,go back 3,house serve 2,steak really 2,portion veggies 2,steak medium 2,steak medium rare 2,E8CBQWDoCK3WujyMFfVG2A
all,nothing special 2,brewery district 2,actually get 1,recommend kazuko ramen 1,restaurants probably times time 1,restaurants probably times 1,restaurants probably 1,restaurant left make room next 1,restaurant left make room 1,restaurant left make 1,...,restaurant brewery district actually 1,restaurant brewery district 1,restaurant brewery 1,recommend kazuko 1,room next 1,really enjoyed 1,ramen tokiwa since love trying 1,ramen tokiwa since love 1,ramen tokiwa since 1,8wjRidyw7pqepAf5MncA-g
all,mixed greens 2,place used 2,beef mixed 2,beef mixed greens 2,fried rice 2,chicken fried rice 2,low brow 2,chicken fried 2,used order 2,egg roll 2,...,price went suppose must 1,really anyone matter karaoke friday 1,price went suppose 1,p

In [67]:
data.business_id.nunique()

C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1935